In [1]:
import re
import pandas as pd

MBO_path = 'res/MBO.txt'
TOP_path = 'res/TOP.txt'
MBL_path = 'res/MBL.txt'

def parse_txt(file_path):
    with open(file_path, 'r') as f:
        full_txt = f.read().splitlines()
    full_txt = ''.join(['\n' if x == '' else x for x in full_txt]).split('\n')

    df = pd.DataFrame()
    for x in full_txt[2:-2]:

        txt = x[x.find('{')+1:x.rfind('}')]
        txt = txt.replace('{    }','{}').replace('{\n    }','{}')
        txt = re.sub('" ','"',txt)
        evt = list(filter(None,re.split(',|=', ','.join(txt.split()))))
        df = pd.concat([df,pd.DataFrame(dict(zip(evt[::2], evt[1::2])),index=[0])],ignore_index=True)

    return df

MBO = parse_txt(MBO_path)
TOP = parse_txt(TOP_path)
MBL = parse_txt(MBL_path)

In [3]:
MBO.to_csv('res/Market_ByOrder.csv',index=False)
TOP.to_csv('res/TopBroker.csv',index=False)
MBL.to_csv('res/Market_ByLevel.csv',index=False)

In [210]:
MBO = MBO[['MKTDEPTH_EVENT_SUBTYPE','MD_TABLE_CMD_RT','MBO_SEQNUM_RT','MBO_ASK_POSITION_RT','MBO_ASK_RT','MBO_ORDER_ID_RT','MBO_ASK_SIZE_RT','MBO_TIME_RT','MBO_BID_POSITION_RT','MBO_BID_RT','MBO_BID_SIZE_RT']]
TOP = TOP[['MKTDEPTH_EVENT_SUBTYPE','MD_TABLE_CMD_RT','MBO_TIME_RT','MBO_BID_POSITION_RT','MBO_BID_RT','MBO_BID_BROKER_RT','MBO_ASK_POSITION_RT','MBO_ASK_RT','MBO_ASK_BROKER_RT']]

In [5]:
TOP = TOP[['MBO_TIME_RT','MBO_BID_POSITION_RT','MBO_BID_RT','MBO_BID_BROKER_RT','MBO_ASK_POSITION_RT','MBO_ASK_RT','MBO_ASK_BROKER_RT']]
TOP

,MBO_TIME_RT,MBO_BID_POSITION_RT,MBO_BID_RT,MBO_BID_BROKER_RT,MBO_ASK_POSITION_RT,MBO_ASK_RT,MBO_ASK_BROKER_RT
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-08-12T01:00:01.000+00:00,1,1.26000022888184,"""1978""",NaN,NaN,NaN
2,2021-08-12T01:00:04.000+00:00,2,1.23000049591064,"""6699""",NaN,NaN,NaN
3,2021-08-12T01:00:04.000+00:00,3,1.22000026702881,"""6699""",NaN,NaN,NaN
4,2021-08-12T01:00:10.000+00:00,4,1.15000057220459,"""8730""",NaN,NaN,NaN
...,...,...,...,...,...,...,...
1375,2021-08-12T08:05:34.000+00:00,17,1.19999980926514,"""4663""",NaN,NaN,NaN
1376,2021-08-12T08:05:34.000+00:00,18,1.15999984741211,"""6699""",NaN,NaN,NaN
1377,2021-08-12T08:05:34.000+00:00,19,1.15000057220459,"""8730""",NaN,NaN,NaN
1378,2021-08-12T08:05:34.000+00:00,20,1.11999988555908,"""6729""",NaN,NaN,NaN


In [7]:
MBO = MBO[['MBO_TIME_RT','MBO_ASK_POSITION_RT','MBO_ASK_RT','MBO_ASK_SIZE_RT','MBO_BID_POSITION_RT','MBO_BID_RT','MBO_BID_COND_CODE_RT','MBO_BID_SIZE_RT']]
MBO_ask = MBO.dropna(subset=['MBO_ASK_POSITION_RT'])

In [23]:
TOP[(TOP['MBO_ASK_POSITION_RT'] == '4') & (TOP['MBO_ASK_RT'] == '1.39999961853027')]

,MBO_TIME_RT,MBO_BID_POSITION_RT,MBO_BID_RT,MBO_BID_BROKER_RT,MBO_ASK_POSITION_RT,MBO_ASK_RT,MBO_ASK_BROKER_RT
53,2021-08-12T01:30:03.000+00:00,NaN,NaN,NaN,4,1.39999961853027,"""8592"""
131,2021-08-12T01:35:01.000+00:00,NaN,NaN,NaN,4,1.39999961853027,"""8592"""
136,2021-08-12T01:35:01.000+00:00,NaN,NaN,NaN,4,1.39999961853027,"""8592"""


In [21]:
MBO.iloc[17]

MBO_TIME_RT             2021-08-12T01:30:03.118+00:00
MBO_ASK_POSITION_RT                                 4
MBO_ASK_RT                           1.39999961853027
MBO_ASK_SIZE_RT                                 10000
MBO_BID_POSITION_RT                               NaN
MBO_BID_RT                                        NaN
MBO_BID_COND_CODE_RT                              NaN
MBO_BID_SIZE_RT                                   NaN
Name: 17, dtype: object

In [24]:
print(f'TOP Timestamp: {TOP.iloc[53]["MBO_TIME_RT"]}')
print(f'MBO Timestamp: {MBO.iloc[17]["MBO_TIME_RT"]}')

TOP Timestamp: 2021-08-12T01:30:03.000+00:00
MBO Timestamp: 2021-08-12T01:30:03.118+00:00


Steps:

1. Start with MBO database. 

If there are new "ADD" to the book, go to TOP and see who the broker is. 

If there are "DEL", check if there are either trades or just pulled. All the brokers order after him have to be manually adjusted (i.e. order -=1)

If there are "MOD", either partially filled or order modified.
